# **Ensemble Learning**

### **Student Identification**

Student Name       | Student Email
-------------------|------------------
Daniel Branco      | r20191230@novaims.unl.pt
Filipe Dias        | r20181050@novaims.unl.pt
Gonçalo Lourenço   | r20191097@novaims.unl.pt
Inês Santos        | r20191184@novaims.unl.pt
Manuel Marreiros   | r20191223@novaims.unl.pt

### **Data Source**

Brain Tumor Classification (MRI) Dataset: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri

Drive with data: https://drive.google.com/file/d/1P3hcUss5Kqb28_WQUcTsuTW2VjNTX4pd/view?usp=share_link

### **Notebook Summary**





In this notebook we applied a technique called **ensemble learning**. The ensemble of classifiers seeks better predictive performance by combining the predictions from multiple models. In fact, it is proven that an ensemble of classifiers is typically more accurate than a single classifier, which makes this a very important tool in classification tasks.

Here, we will try to merge some of our models (namely convolutional neural networks 4, 5 and 6) and see how the performance compares to the individual models.

### **References**

1. [Ensemble deep learning: A review
](https://arxiv.org/pdf/2104.02395.pdf)

### **Imports**

In [ ]:
pip install keras-tuner tensorflow-addons --quiet

In [ ]:
import os
import time
import math
import random 
import zipfile
import shutil

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.colors import ListedColormap

import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, Model, layers, initializers, regularizers, optimizers, metrics 
from tensorflow.keras.initializers import GlorotNormal
import tensorflow_addons as tfa

import keras
from keras_tuner import Objective
import keras_tuner as kt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

### **Things needed from previous notebooks**

In [ ]:
#EXPLORATION

# Set the machine
gdrive = True
# Set the connection string
path = "/content/drive/MyDrive/Deep Learning/Projeto/"
main_folder, training_folder, testing_folder = "brain_tumor_data/", "Training/", "Testing/"
# If using Google Drive
if gdrive:
    # Setup drive
    from google.colab import drive
    drive.mount('/content/drive')        
    # Transfer zip dataset to the current virtual machine
    t0 = time.time()
    shutil.copyfile(path + 'brain_tumor_data.zip', 'brain_tumor_data.zip')
    # Extract files
    zip_ = zipfile.ZipFile('brain_tumor_data.zip')
    zip_.extractall()
    zip_.close()
    print("File transfer completed in %0.3f seconds" % (time.time() - t0))
    path = ""

classes = ["no_tumor", "glioma_tumor", "meningioma_tumor", "pituitary_tumor"]

# Create empty lists to store the number of instances and class names
n_train = []
class_names = []

# Loop through each class in the dataset
for c in classes:
    # Get the number of instances in the training set for the current class
    n_train_c = len(os.listdir(path + main_folder + training_folder + f"/{c}"))
    # Append the number of instances and class name to their respective lists
    n_train.append(n_train_c)
    class_names.append(c)

image_size=(128, 128)
crop_to_aspect_ratio=True
color_mode='grayscale'
batch_size=64
label_mode="categorical"
validation_split=0.2
shuffle=True
seed=0

ds_train, ds_val = image_dataset_from_directory(path + main_folder + training_folder, 
                                                image_size=image_size,
                                                crop_to_aspect_ratio=crop_to_aspect_ratio,
                                                color_mode=color_mode,
                                                batch_size=batch_size,
                                                label_mode=label_mode,
                                                subset='both',
                                                validation_split=validation_split, 
                                                shuffle=shuffle,
                                                seed=seed)

iter_train = iter(ds_train)
batch_x_train, batch_y_train = iter_train.next()

n_classes = len(classes)
total_samples = np.sum(n_train)

#PREPROCESSING

class_weights = {}
for i in range(n_classes):
    w = total_samples / (2.0 * n_train[i])
    class_weights[i] = w

print('Class counts:', n_train)
print('Class weights:', class_weights)

input_shape = tuple(batch_x_train.shape)
rescaling = layers.Rescaling(1./255)
batchnormalization = layers.BatchNormalization()

rotation_layer = layers.RandomRotation(factor=0.05)
zoom_layer = layers.RandomZoom(height_factor=0.05, width_factor=0.05)
contrast_layer = layers.RandomContrast(factor=0.10)
brightness_layer = layers.RandomBrightness(factor=0.05)
noise_layer = layers.GaussianNoise(0.05)
flip_layer = layers.RandomFlip(mode='horizontal')
crop_layer = layers.RandomCrop(height=300, width=300)
translation_layer = layers.RandomTranslation(height_factor=0.1, width_factor=0.1)

def augmentation(inputs):
    x = rotation_layer(inputs)
    x = zoom_layer(x)
    x = contrast_layer(x)
    x = brightness_layer(x)
    x = noise_layer(x)
    return x

#MODEL HANDCRAFTED
class CNN4(tf.keras.Model):
    def __init__(self, seed=0):
        super().__init__()
        self.augmentation = augmentation
        self.batchnormalization = layers.BatchNormalization()
        self.conv1 = layers.Conv2D(filters=32*input_shape[-1], kernel_size=(3, 3), kernel_initializer=initializers.GlorotNormal(seed=seed))
        self.relu = layers.Activation("relu")
        self.maxpool1 = layers.MaxPooling2D(pool_size=(2, 2))          
        self.conv2 = layers.Conv2D(filters=64*input_shape[-1], kernel_size=(3, 3), kernel_initializer=initializers.GlorotNormal(seed=seed))
        self.maxpool2 = layers.MaxPooling2D(pool_size=(2, 2))      
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(units=4, activation="softmax", kernel_initializer=initializers.GlorotNormal(seed=seed)) 
        
    def call(self, inputs):
        x = self.augmentation(inputs)
        x = self.batchnormalization(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return x

class CNN5(tf.keras.Model):
    def __init__(self, seed=0):
        super().__init__()
        self.augmentation = augmentation
        self.batchnormalization = layers.BatchNormalization()
        self.conv1 = layers.Conv2D(filters=32*input_shape[-1], kernel_size=(3, 3), kernel_initializer=initializers.GlorotNormal(seed=seed))
        self.relu = layers.Activation("relu")
        self.maxpool = layers.MaxPooling2D(pool_size=(2, 2))          
        self.conv2 = layers.Conv2D(filters=64*input_shape[-1], kernel_size=(3, 3), kernel_initializer=initializers.GlorotNormal(seed=seed))
        self.dropout = layers.Dropout(0.3) #rate at which input units are set to 0
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(units=4, activation="softmax", kernel_initializer=initializers.GlorotNormal(seed=seed)) 
        
    def call(self, inputs):
        x = self.augmentation(inputs)
        x = self.batchnormalization(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return x

class CNN6(tf.keras.Model):
    def __init__(self, seed=0):
        super().__init__()
        self.augmentation = augmentation
        self.batchnormalization = layers.BatchNormalization()
        self.conv1 = layers.Conv2D(
            filters=32*input_shape[-1], 
            kernel_size=(3, 3), 
            kernel_initializer=initializers.GlorotNormal(seed=seed),
            kernel_regularizer=regularizers.l2(0.001), # Adding kernel_regularizer
        )
        self.relu = layers.Activation("relu")
        self.maxpool = layers.MaxPooling2D(pool_size=(2, 2))          
        self.conv2 = layers.Conv2D(
            filters=64*input_shape[-1], 
            kernel_size=(3, 3), 
            kernel_initializer=initializers.GlorotNormal(seed=seed),
            kernel_regularizer=regularizers.l2(0.001), # Adding kernel_regularizer
        )
        self.dropout = layers.Dropout(0.3) #rate at which input units are set to 0
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(
            units=4, 
            activation="softmax", 
            kernel_initializer=initializers.GlorotNormal(seed=seed),
            kernel_regularizer=regularizers.l2(0.001), # Adding kernel_regularizer
            activity_regularizer=regularizers.l1(0.001) # Adding activity_regularizer
        ) 
        
    def call(self, inputs):
        x = self.augmentation(inputs)
        x = self.batchnormalization(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return x





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File transfer completed in 1.896 seconds
Found 2870 files belonging to 4 classes.
Using 2296 files for training.
Using 574 files for validation.


## **Ensemble of Classifiers**

Let's start by creating, compiling and training three of our best models.

In [ ]:
cnn4 = CNN4(seed=seed)
cnn5 = CNN4(seed=seed)
cnn6 = CNN4(seed=seed)

In [ ]:
cnn4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.CategoricalAccuracy(name='accuracy'),
        tfa.metrics.F1Score(num_classes=4, average='macro', name='F1-Score')])
cnn4.fit(ds_train, epochs=epochs, validation_data=ds_val, class_weight = class_weights)

cnn5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.CategoricalAccuracy(name='accuracy'),
        tfa.metrics.F1Score(num_classes=4, average='macro', name='F1-Score')])
cnn5.fit(ds_train, epochs=epochs, validation_data=ds_val, class_weight = class_weights)

cnn6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.CategoricalAccuracy(name='accuracy'),
        tfa.metrics.F1Score(num_classes=4, average='macro', name='F1-Score')])
cnn6.fit(ds_train, epochs=epochs, validation_data=ds_val, class_weight = class_weights)


In [ ]:
from numpy import argmax

# Predict the test set using each model
y_pred_1 = cnn4.predict(ds_test)
y_pred_2 = cnn5.predict(ds_test)
y_pred_3 = cnn6.predict(ds_test)

# Combine the predictions
y_pred_ensemble = argmax(y_pred_1 + y_pred_2 + y_pred_3, axis=1)


7/7 [==============================] - 0s 10ms/step


Firstly, the test set ds_test is passed through each of the three CNN models, and the predicted classes are obtained for each input image. The predictions are stored in y_pred_1, y_pred_2, and y_pred_3, respectively.

Then, the ensemble prediction is performed by combining the individual predictions from the three models, which returns the most frequent value.

Finally, the resulting ensemble predictions are stored in y_pred_ensemble, which contains the predicted class labels for each input image in the test set, based on the combined output of the three CNN models.

In [ ]:
print(y_pred_ensemble)

[1 2 2 2 2 2 1 2 2 1 1 2 1 2 2 3 2 1 2 1 2 2 2 2 2 1 2 2 2 0 1 2 2 2 1 1 2
 2 3 2 2 1 2 1 2 1 3 2 3 2 0 2 2 2 2 2 0 1 2 1 1 2 0 2 2 1 2 2 2 1 3 2 2 2
 2 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 1 2 1 3 2 2 2 2 1 1 2 2 2 2 3 1 1 2 2
 2 2 1 2 2 2 3 0 1 2 2 2 0 1 2 2 2 3 2 2 2 2 2 2 1 1 2 2 2 3 2 2 2 2 1 2 2
 2 2 2 3 1 0 2 1 2 1 2 2 2 1 2 2 3 2 2 2 2 2 2 2 2 3 3 2 2 2 1 1 2 2 2 1 1
 2 1 2 2 2 3 2 2 1 2 2 2 3 2 2 1 3 2 3 1 2 1 1 2 0 2 2 2 2 2 3 1 2 2 1 3 2
 1 2 2 3 3 1 1 2 2 2 3 1 2 3 3 3 1 1 3 2 0 2 2 2 3 2 2 2 3 0 3 2 1 1 2 2 2
 1 1 2 3 2 2 3 3 1 2 1 2 2 2 2 2 2 0 1 1 2 1 2 3 1 2 2 2 2 1 3 1 2 1 1 1 1
 2 3 1 3 2 2 2 3 2 2 1 2 1 2 2 1 2 3 2 2 1 3 3 2 3 1 0 2 1 1 1 2 1 1 2 2 2
 2 1 3 2 0 2 3 1 2 2 1 2 2 1 3 2 3 2 2 3 2 3 2 1 1 1 2 1 3 2 2 2 1 2 3 2 2
 2 2 3 1 2 1 3 1 2 3 3 1 2 1 0 2 0 2 2 1 2 3 2 2]
